In [143]:
#!pip install pql
import pql

testAll = pql.find("elemets == all(['Si', 'O', 3])")

In [144]:
pql.find("formula_prototype=='AB2C3' and nelements==4 and elements==all(['Si', 'O'])")
pql.find("x<=3 ")
pql.find("(x<=3  and  y!=2)  or  not z<0")

{'$or': [{'$and': [{'x': {'$lte': 3}}, {'y': {'$ne': 2}}]},
  {'z': {'$not': {'$lt': 0}}}]}

In [418]:
def removeFilter(raw):
    if(raw.find("filter =") != -1):
        return raw[len("filter ="):]
    elif(raw.find("filter=") != -1):
        return raw[len("filter="):]
    else:
        return raw
    
def insert(original, index, insertion):
    result = original[:index] + insertion + original[index:]
    return result

def onlyEqual(string, index):
    return string[index] == "=" and not (string[index-1] == "<" 
                     or string[index+1] == ">" 
                     or string[index-1] == "=" 
                     or string[index+1] == "=") 


def combineMultiple(string, index):
    firstQuoteIndex = 0
    for firstQuoteIndex in reversed(range(0, index)):
        if(ord(string[firstQuoteIndex]) == 34):
            break
    lastQuoteIndex = index
    for lastQuoteIndex in range(index, len(string)):
        if(ord(string[lastQuoteIndex])== 34):
            break
    insertion = string[firstQuoteIndex+1:lastQuoteIndex]
    insertion = insertion.split(",")
    return string[:firstQuoteIndex] + 'all({})'.format(insertion) + string[lastQuoteIndex+1:]

In [433]:
"""
Parsing the input(Optimade form) to Materials Project's MongoDB Query

ex 1: (change "=" to ==,  "" to '', and "Si, O" to all(['Si', 'O']))
input -> filter=formula_prototype="AB2C3"  AND  nelements=4  AND  elements="Si,O"
output ->  formula_prototype='AB2C3' and nelements==4 and elements==all(['Si', 'O'])

ex2:(no change)
input -> filter=(x<3  and  y!=2)  or  not  z<0
output -> (x<3  and  y!=2)  or  not  z<0

"""
def parseInput(raw):
    filterRemoved = removeFilter(raw)
    lowerCased = filterRemoved.replace("AND","and").replace("OR", "or")
    result = lowerCased
    length = len(result)
    i = 0
    while(i < length):
        if(onlyEqual(result, i)):
            result = insert(result, i, "=")
            length = len(result)
        if(result[i] == ","):
            result = combineMultiple(result, i)
        i = i + 1 
    return result

In [435]:
x = parseInput('filter=formula_prototype="AB2C3" and nelements=4 AND elements="Si,O"')
print(x)
pql.find(x)

im here
formula_prototype=="AB2C3" and nelements==4 and elements==all(['Si', 'O'])


{'$and': [{'formula_prototype': 'AB2C3'},
  {'nelements': 4},
  {'elements': {'$all': ['Si', 'O']}}]}